In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/rtl.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))


# فصل ۲ — مبانی داده و پیش‌پردازش
## درس ۴: مقیاس‌بندی ویژگی‌ها (نرمال‌سازی و استانداردسازی)

### در این درس چه چیزهایی یاد می‌گیرید؟

در پایان این نوت‌بوک می‌توانید:

- توضیح دهید *چرا* مقیاس‌بندی ویژگی‌ها مهم است و *چه زمانی* اهمیت چندانی ندارد.
- بین **استانداردسازی** (مقیاس z-score) و **نرمال‌سازی** (مقیاس min–max) و همچنین **نرمال‌سازی برداری** (یکه‌سازی طول بردار برای هر نمونه) تفاوت قائل شوید.
- برای هر الگوریتم، اسکیلر مناسب را انتخاب کنید (kNN / SVM / رگرسیون لجستیک / PCA / k-means / مدل‌های خطی با منظم‌سازی).
- مقیاس‌بندی را به‌درستی و **بدون نشت داده (data leakage)** با استفاده از `Pipeline` و `ColumnTransformer` پیاده‌سازی کنید.
- مشکلات رایج مقیاس‌بندی را تشخیص دهید: داده‌های پرت، دُم‌های سنگین، ماتریس‌های تنک (sparse)، و انواع داده‌ی ترکیبی.
- اسکیلر را مثل یک **هایپرپارامتر** ببینید و با cross-validation / grid search اعتبارسنجی کنید.
- تصمیم‌های مربوط به مقیاس‌بندی را در چارچوب یک روند قابل بازتولید در ML مستندسازی کنید.

---

### چرا این موضوع برای سطح پیشرفته (Band 8–9) مهم است؟

در سطح بالا، «Feature Scaling» یک کار مکانیکی نیست؛ یک *تصمیم مدل‌سازی* است که روی موارد زیر اثر می‌گذارد:

- **هندسه‌ی بهینه‌سازی** (Conditioning مسئله و اندازه‌ی گام‌ها در گرادیان‌کاهشی).
- **معنای منظم‌سازی** (جریمه‌های L1/L2 بدون کنترل مقیاس بین ویژگی‌ها قابل مقایسه نیستند).
- **فاصله و شباهت** (kNN، k-means، روش‌های کرنلی، فاصله‌ی کسینوسی).
- **پایداری عددی** (دامنه‌ی اعداد شناور؛ ویژگی‌های بد‌مقیاس می‌توانند باعث underflow/overflow یا ماتریس بدشرط شوند).
- **تفسیرپذیری و حاکمیت مدل** (اگر یکی از ویژگی‌ها دلار باشد و دیگری میلی‌متر، ضریب مدل چه مفهومی دارد؟)

در این درس یاد می‌گیرید مقیاس‌بندی را به‌عنوان بخشی از خط لوله‌ی مدل‌سازی ببینید، نه یک ترفند جداگانه.


In [2]:

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 160)

print("Versions:")
import sklearn, sys
print("  python:", sys.version.split()[0])
print("  sklearn:", sklearn.__version__)


Versions:
  python: 3.13.0
  sklearn: 1.5.2



## ۱) مفاهیم پایه و نمادگذاری

فرض کنید داده‌ای با $n$ نمونه و $p$ ویژگی داریم. بردار ویژگی برای نمونه‌ی $i$ را به‌شکل زیر می‌نویسیم:

$$
\mathbf{x}_i = [x_{i1}, x_{i2}, \dots, x_{ip}]
$$

دو عملیات مقیاس‌بندی که دائماً با آن‌ها سروکار دارید:

### استانداردسازی (مقیاس z-score)

برای ویژگی $j$:

$$
z_{ij} = \frac{x_{ij} - \mu_j}{\sigma_j}
$$

- $\mu_j$ میانگین ویژگی $j$ است (فقط بر اساس داده‌ی **آموزش** محاسبه می‌شود).
- $\sigma_j$ انحراف معیار ویژگی $j$ است (فقط بر اساس داده‌ی **آموزش** محاسبه می‌شود).

### نرمال‌سازی min–max

برای ویژگی $j$:

$$
x'_{ij} = \frac{x_{ij} - \min_j}{\max_j - \min_j}
$$

این کار هر ویژگی را (تقریباً) به بازه‌ی $[0, 1]$ نگاشت می‌کند.

### نرمال‌سازی برداری (یکه‌سازی طول بردار برای هر نمونه)

در این روش، کل بردار ویژگی یک نمونه نرمال می‌شود:

$$
\tilde{\mathbf{x}}_i = \frac{\mathbf{x}_i}{\lVert \mathbf{x}_i \rVert_2}
$$

این روش در بازیابی متن / شباهت کسینوسی رایج است و با min–max تفاوت دارد.

---

## ۲) چه زمانی مقیاس‌بندی مهم است (و چه زمانی نیست)

مقیاس‌بندی زمانی مهم است که الگوریتم شما از موارد زیر استفاده کند:

- **فاصله‌ها یا ضرب داخلی**: kNN، k-means، SVM (به‌ویژه کرنل RBF)، PCA، Kernel Ridge.
- **جریمه‌های منظم‌سازی** که فرض می‌کنند مقیاس ضرایب قابل مقایسه است: Lasso، Ridge، Elastic Net.
- **بهینه‌سازی مختصات‌به‌مختصات** که اندازه‌ی گام‌ها به مقیاس ویژگی وابسته است.

مقیاس‌بندی معمولاً برای موارد زیر ضروری نیست:

- **مدل‌های مبتنی بر درخت**: درخت تصمیم، جنگل تصادفی، گرادیان بوستینگ درختی  
  (تقسیم‌ها بر اساس ترتیب هستند؛ مقیاس ترتیب را تغییر نمی‌دهد).
- برخی سامانه‌های **قاعده‌محور** یا **شمارشی** که ویژگی‌ها از ابتدا هم‌مقیاس هستند.

با این حال، «ضروری نبودن» به معنای «هرگز مفید نبودن» نیست. مدل‌های درختی ممکن است به‌طور غیرمستقیم سود ببرند (مثلاً وقتی PCA یا سایر گام‌های قبلی به مقیاس حساس‌اند).

---

### شهود هندسی (غلبه‌ی یک ویژگی بر فاصله)

فاصله‌ی اقلیدسی را در نظر بگیرید:

$$
d(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_{j=1}^{p} (x_j - y_j)^2}
$$

اگر یک ویژگی در بازه‌ی $[0, 10^6]$ و دیگری در بازه‌ی $[0, 1]$ باشد، ویژگی بزرگ‌دامنه تقریباً همیشه *بر فاصله غالب می‌شود*؛ حتی اگر اطلاعات چندانی نداشته باشد. مقیاس‌بندی در عمل یعنی تعریف این‌که «فاصله» برای مسئله‌ی شما چه معنایی دارد.



## ۳) چند دیتاست را بارگذاری می‌کنیم (و مقیاس ویژگی‌ها را می‌بینیم)

در پروژه‌های واقعی، مقیاس‌بندی معمولاً «یک دیتاست، یک اسکیلر» نیست. اغلب یک سیاست قابل استفاده‌ی مجدد تعریف می‌کنید و سپس آن را روی منابع داده‌ی مختلف اعتبارسنجی می‌کنید.

در این نوت‌بوک از چند دیتاست موجود در ریپوی شما استفاده می‌کنیم:

- `diabetes.csv` (طبقه‌بندی دودویی؛ ویژگی‌های عددی با دامنه‌های متفاوت)
- `iris.csv` (طبقه‌بندی چندکلاسه؛ نمونه‌ی کلاسیک برای روش‌های مبتنی بر فاصله)
- `Wine_Quality.csv` (داده‌ی جدولی؛ برای سادگی کیفیت را دودویی می‌کنیم)
- `drug200.csv` (ترکیبی از عددی و دسته‌ای؛ برای نمایش Pipeline ستونی)
- `hw_200.csv` (خوشه‌بندی؛ برای نمایش اثر مقیاس‌بندی بر k-means و PCA)

ابتدا دامنه‌ی خام ویژگی‌های عددی را بررسی می‌کنیم.


In [3]:

# Paths are relative to: Tutorials/English/Chapter2 or Tutorials/Persian/Chapter2
p_diabetes = Path("../../../Datasets/Classification/diabetes.csv")
p_iris     = Path("../../../Datasets/Classification/iris.csv")
p_wine     = Path("../../../Datasets/Classification/Wine_Quality.csv")
p_drug     = Path("../../../Datasets/Classification/drug200.csv")
p_hw       = Path("../../../Datasets/Clustering/hw_200.csv")

diabetes = pd.read_csv(p_diabetes)
iris = pd.read_csv(p_iris)
wine = pd.read_csv(p_wine)
drug = pd.read_csv(p_drug)
hw_raw = pd.read_csv(p_hw)

print("Loaded shapes:")
print("  diabetes:", diabetes.shape)
print("  iris:", iris.shape)
print("  wine:", wine.shape)
print("  drug:", drug.shape)
print("  hw_raw:", hw_raw.shape)

display(diabetes.head())


Loaded shapes:
  diabetes: (768, 9)
  iris: (150, 5)
  wine: (4898, 12)
  drug: (200, 6)
  hw_raw: (200, 3)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,Non-Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,Non-Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic


In [4]:

def scale_summary(df: pd.DataFrame, name: str, max_cols: int = 10):
    num = df.select_dtypes(include=[np.number])
    if num.empty:
        print(f"{name}: no numeric columns")
        return
    s = pd.DataFrame({
        "min": num.min(),
        "max": num.max(),
        "mean": num.mean(),
        "std": num.std(ddof=0),
    })
    s["range"] = s["max"] - s["min"]
    s["range_ratio_to_median"] = s["range"] / (np.median(s["range"]) + 1e-12)
    s = s.sort_values("range", ascending=False)
    print(f"\n{name}: numeric scale summary (top by range)")
    display(s.head(max_cols).round(4))

scale_summary(diabetes, "diabetes")
scale_summary(iris, "iris")
scale_summary(wine, "wine")



diabetes: numeric scale summary (top by range)


,min,max,mean,std,range,range_ratio_to_median
Insulin,0.000,846.00,79.7995,115.1689,846.000,10.1866
Glucose,0.000,199.00,120.8945,31.9518,199.000,2.3961
BloodPressure,0.000,122.00,69.1055,19.3432,122.000,1.4690
SkinThickness,0.000,99.00,20.5365,15.9418,99.000,1.1921
BMI,0.000,67.10,31.9926,7.8790,67.100,0.8079
Age,21.000,81.00,33.2409,11.7526,60.000,0.7225
Pregnancies,0.000,17.00,3.8451,3.3674,17.000,0.2047
DiabetesPedigreeFunction,0.078,2.42,0.4719,0.3311,2.342,0.0282



iris: numeric scale summary (top by range)


,min,max,mean,std,range,range_ratio_to_median
petal_length,1.0,6.9,3.7587,1.7585,5.9,1.9667
sepal_length,4.3,7.9,5.8433,0.8253,3.6,1.2000
sepal_width,2.0,4.4,3.0540,0.4321,2.4,0.8000
petal_width,0.1,2.5,1.1987,0.7606,2.4,0.8000



wine: numeric scale summary (top by range)


,min,max,mean,std,range,range_ratio_to_median
total sulfur dioxide,9.00,440.00,138.3607,42.4937,431.00,112.5326
free sulfur dioxide,2.00,289.00,35.3081,17.0054,287.00,74.9347
residual sugar,0.60,65.80,6.3914,5.0715,65.20,17.0235
fixed acidity,3.80,14.20,6.8548,0.8438,10.40,2.7154
alcohol,8.00,14.20,10.5143,1.2305,6.20,1.6188
quality,3.00,9.00,5.8779,0.8855,6.00,1.5666
citric acid,0.00,1.66,0.3342,0.1210,1.66,0.4334
pH,2.72,3.82,3.1883,0.1510,1.10,0.2872
volatile acidity,0.08,1.10,0.2782,0.1008,1.02,0.2663
sulphates,0.22,1.08,0.4898,0.1141,0.86,0.2245



### تفسیر

حتی اگر همه‌ی ستون‌ها «عددی» به نظر برسند، مقیاس‌ها یکسان نیستند:

- در `diabetes`، ویژگی‌های **Insulin** و **Glucose** در دامنه‌ای بسیار بزرگ‌تر از **DiabetesPedigreeFunction** قرار دارند.
- در `wine`، برخی شاخص‌های شیمیایی پراکندگی‌های بسیار متفاوتی دارند (مثلاً sulphates در برابر free sulfur dioxide).
- در `iris` دامنه‌ها متوسط است، اما مقیاس‌بندی همچنان می‌تواند هندسه‌ی همسایگی را تغییر دهد.

از همین‌جا تصمیم‌گیری درباره‌ی مقیاس‌بندی شروع می‌شود.



## ۴) استانداردسازی در عمل (z-score) — و این‌که چرا به بهینه‌سازی کمک می‌کند

بسیاری از الگوریتم‌ها در عمل یک مسئله‌ی بهینه‌سازی را حل می‌کنند. برای مثال، رگرسیون لجستیک (با منظم‌سازی L2) می‌تواند این‌گونه نوشته شود:

$$
\min_{\mathbf{w}, b} \; \frac{1}{n}\sum_{i=1}^n \log\left(1 + \exp\left(-y_i(\mathbf{w}^\top \mathbf{x}_i + b)\right)\right) + \lambda \lVert \mathbf{w} \rVert_2^2
$$

اگر یک ویژگی ۱۰۰۰ برابر بزرگ‌تر از دیگری باشد، سطح تابع هزینه **بدشرط (ill-conditioned)** می‌شود؛ در نتیجه بهینه‌ساز ممکن است در یک جهت گام‌های بسیار کوچک و در جهت دیگر گام‌های بزرگ نیاز داشته باشد. استانداردسازی مسئله را به حالتی نزدیک‌تر به «کروی» می‌برد و رفتار همگرایی را بهبود می‌دهد.

### مثال: رگرسیون لجستیک روی دیتاست diabetes

مقایسه می‌کنیم:

- مدل A: رگرسیون لجستیک **بدون** مقیاس‌بندی
- مدل B: رگرسیون لجستیک با `StandardScaler` داخل یک `Pipeline`

ارزیابی را روی یک تقسیم آموزش/آزمون انجام می‌دهیم.


In [5]:

from sklearn.preprocessing import LabelEncoder

X = diabetes.drop(columns=["classification"])
y = diabetes["classification"].copy()

le = LabelEncoder()
y_bin = le.fit_transform(y)
print("Classes:", list(le.classes_))

X_train, X_test, y_train, y_test = train_test_split(
    X, y_bin, test_size=0.25, random_state=42, stratify=y_bin
)

# A) No scaling
lr_raw = LogisticRegression(max_iter=2000, solver="lbfgs")
lr_raw.fit(X_train, y_train)
pred_raw = lr_raw.predict(X_test)

# B) With standardization
lr_scaled = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000, solver="lbfgs"))
])
lr_scaled.fit(X_train, y_train)
pred_scaled = lr_scaled.predict(X_test)

print("\nAccuracy (no scaling):", round(accuracy_score(y_test, pred_raw), 4))
print("Accuracy (standardized):", round(accuracy_score(y_test, pred_scaled), 4))

print("\nConfusion matrix (no scaling):\n", confusion_matrix(y_test, pred_raw))
print("\nConfusion matrix (standardized):\n", confusion_matrix(y_test, pred_scaled))


Classes: ['Diabetic', 'Non-Diabetic']

Accuracy (no scaling): 0.7812
Accuracy (standardized): 0.7865

Confusion matrix (no scaling):
 [[ 39  28]
 [ 14 111]]

Confusion matrix (standardized):
 [[ 40  27]
 [ 14 111]]



Accuracy (no scaling): 0.4933
Accuracy (standardized): 0.5

Confusion matrix (no scaling):
 [[59 23]
 [53 15]]

Confusion matrix (standardized):
 [[63 19]
 [56 12]]



### ضرایب و تفسیرپذیری: مقیاس‌بندی معنای ضرایب را تغییر می‌دهد

اگر یک مدل خطی را *بدون* مقیاس‌بندی برازش کنید، اندازه‌ی ضرایب به واحد اندازه‌گیری ویژگی‌ها وابسته می‌شود.

استانداردسازی ضرایب را قابل مقایسه‌تر می‌کند؛ یعنی «اثر به ازای یک انحراف معیار» که در بسیاری از کاربردها به شهود *اهمیت نسبی* نزدیک‌تر است.

بیایید ضرایب مدل استانداردشده را ببینیم.


In [6]:

feature_names = X.columns.tolist()

coef = lr_scaled.named_steps["clf"].coef_.ravel()
coef_s = pd.Series(coef, index=feature_names).sort_values(key=lambda s: np.abs(s), ascending=False)

display(coef_s.to_frame("coef (standardized)").head(10).round(4))
print("Interpretation: coefficients after StandardScaler are roughly 'effect per 1 std' of the feature.")


,coef (standardized)
Glucose,-1.1240
BMI,-0.6716
Pregnancies,-0.4667
BloodPressure,0.3129
DiabetesPedigreeFunction,-0.2806
Insulin,0.1773
Age,-0.1613
SkinThickness,-0.1089


Interpretation: coefficients after StandardScaler are roughly 'effect per 1 std' of the feature.



## ۵) نرمال‌سازی min–max در برابر استانداردسازی — ملاحظات عملی

### چه زمانی min–max مفید است؟

مقیاس min–max رایج است وقتی:

- می‌خواهید ویژگی‌ها در بازه‌ی محدود $[0, 1]$ قرار بگیرند.
- مدل یا قیود/پیش‌فرض‌ها انتظار ورودی‌های محدود دارند.
- می‌خواهید فاصله‌های نسبی حفظ شود اما دامنه کنترل گردد.

### چه زمانی استانداردسازی مقاوم‌تر است؟

استانداردسازی معمولاً بهتر کار می‌کند وقتی:

- ویژگی‌ها تقریباً زنگوله‌ای‌اند یا می‌خواهید با آن‌ها چنین برخورد کنید.
- از مدل‌های خطی منظم‌سازی‌شده، SVM یا PCA استفاده می‌کنید.
- مرکز کردن حول صفر اهمیت دارد.

### هشدار مهم (داده‌های پرت)

min–max به داده‌های پرت حساس است: یک مقدار بسیار بزرگ/کوچک می‌تواند کل ویژگی را در بازه‌ای بسیار فشرده قرار دهد. در داده‌های دُم‌سنگین، RobustScaler اغلب انتخاب بهتری است.

---

## ۶) نمایش عملی: kNN روی دیتاست Iris (بدون مقیاس‌بندی vs min–max vs استانداردسازی)

kNN یک روش مبتنی بر فاصله است. اگر یک ویژگی دامنه‌ی بزرگ‌تری داشته باشد، به‌طور ضمنی وزن بیشتری می‌گیرد.

مقایسه می‌کنیم:

- kNN روی ویژگی‌های خام
- kNN با `MinMaxScaler`
- kNN با `StandardScaler`

برای جلوگیری از نشت داده، مقیاس‌بندی را داخل `Pipeline` قرار می‌دهیم.


In [7]:

X = iris.drop(columns=["classification"])
y = iris["classification"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

pipelines = {
    "raw": Pipeline([("knn", KNeighborsClassifier(n_neighbors=7))]),
    "minmax": Pipeline([("scaler", MinMaxScaler()), ("knn", KNeighborsClassifier(n_neighbors=7))]),
    "standard": Pipeline([("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=7))]),
}

for name, pipe in pipelines.items():
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, pred)
    print(f"{name:>8}  accuracy = {acc:.4f}")

print("\nA short classification report for the standardized pipeline:\n")
pred_std = pipelines["standard"].predict(X_test)
print(classification_report(y_test, pred_std))


     raw  accuracy = 0.9474
  minmax  accuracy = 0.9737
standard  accuracy = 0.9474

A short classification report for the standardized pipeline:

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        12
Iris-versicolor       0.87      1.00      0.93        13
 Iris-virginica       1.00      0.85      0.92        13

       accuracy                           0.95        38
      macro avg       0.96      0.95      0.95        38
   weighted avg       0.95      0.95      0.95        38



  minmax  accuracy = 0.9737
standard  accuracy = 0.9474

A short classification report for the standardized pipeline:

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        12
Iris-versicolor       0.87      1.00      0.93        13
 Iris-virginica       1.00      0.85      0.92        13

       accuracy                           0.95        38
      macro avg       0.96      0.95      0.95        38
   weighted avg       0.95      0.95      0.95        38




### بحث (kNN)

در عمل، عملکرد kNN بعد از مقیاس‌بندی می‌تواند تغییر معناداری داشته باشد؛ اما توجه کنید:

- «بهترین اسکیلر» به $k$، معیار فاصله، و دیتاست وابسته است.
- هدف مقیاس‌بندی این نیست که *همیشه* نتیجه را بهتر کند؛ هدف این است که الگوریتم مطابق تعریف مورد انتظار رفتار کند.
- روند درست این است که اسکیلر را مانند یک هایپرپارامتر در نظر بگیرید و اعتبارسنجی کنید.

بیایید این ایده را با cross-validation نشان دهیم.


In [8]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, pipe in pipelines.items():
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="accuracy")
    print(f"{name:>8}  mean={scores.mean():.4f}  std={scores.std():.4f}  scores={np.round(scores,4)}")


     raw  mean=0.9600  std=0.0389  scores=[1.     0.9667 0.9    1.     0.9333]
  minmax  mean=0.9533  std=0.0452  scores=[1.     0.9667 0.9    1.     0.9   ]
standard  mean=0.9600  std=0.0327  scores=[0.9667 0.9667 0.9    1.     0.9667]


  minmax  mean=0.9533  std=0.0452  scores=[1.     0.9667 0.9    1.     0.9   ]


standard  mean=0.9600  std=0.0327  scores=[0.9667 0.9667 0.9    1.     0.9667]



## ۷) داده‌های پرت و دُم‌های سنگین: RobustScaler، PowerTransformer، QuantileTransformer

داده‌های واقعی معمولاً «تمیز و نرمال» نیستند. داده‌های پرت می‌توانند انحراف معیار را ناپایدار کنند و min–max را هم خراب کنند. چند گزینه‌ی مقاوم وجود دارد:

### RobustScaler
از میانه و دامنه‌ی بین‌چارکی (IQR) استفاده می‌کند:

$$
x' = \frac{x - \text{median}(x)}{\text{IQR}(x)}
$$

### PowerTransformer (Yeo–Johnson / Box–Cox)
داده را طوری تبدیل می‌کند که بیشتر شبیه توزیع نرمال شود و گاهی رفتار مدل‌های خطی را بهتر می‌کند.

### QuantileTransformer
بر اساس کوانتایل‌ها داده را به یک توزیع هدف (یکنواخت یا نرمال) نگاشت می‌کند. می‌تواند مفید باشد، اما ممکن است فاصله‌ها را تغییر شکل دهد؛ بنابراین باید با اعتبارسنجی همراه باشد.

نمایش را با دیتاست wine انجام می‌دهیم که برخی ویژگی‌ها می‌توانند چولگی داشته باشند.


In [9]:

wine_y = (wine["quality"] >= 7).astype(int)
wine_X = wine.drop(columns=["quality"])

X_train, X_test, y_train, y_test = train_test_split(
    wine_X, wine_y, test_size=0.25, random_state=42, stratify=wine_y
)

scaler_pipes = {
    "raw": Pipeline([("clf", LogisticRegression(max_iter=4000))]),
    "standard": Pipeline([("scaler", StandardScaler()), ("clf", LogisticRegression(max_iter=4000))]),
    "robust": Pipeline([("scaler", RobustScaler()), ("clf", LogisticRegression(max_iter=4000))]),
    "power_yeojohnson": Pipeline([("scaler", PowerTransformer(method="yeo-johnson", standardize=True)), ("clf", LogisticRegression(max_iter=4000))]),
    "quantile_normal": Pipeline([("scaler", QuantileTransformer(output_distribution="normal", n_quantiles=200, random_state=42)), ("clf", LogisticRegression(max_iter=4000))]),
}

for name, pipe in scaler_pipes.items():
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, pred)
    print(f"{name:>16}  accuracy={acc:.4f}  positive_rate_train={y_train.mean():.3f}")


             raw  accuracy=0.8090  positive_rate_train=0.216
        standard  accuracy=0.8073  positive_rate_train=0.216
          robust  accuracy=0.8024  positive_rate_train=0.216
power_yeojohnson  accuracy=0.8049  positive_rate_train=0.216
 quantile_normal  accuracy=0.8049  positive_rate_train=0.216


        standard  accuracy=0.7367  positive_rate_train=0.266


          robust  accuracy=0.7367  positive_rate_train=0.266


power_yeojohnson  accuracy=0.7367  positive_rate_train=0.266


 quantile_normal  accuracy=0.7367  positive_rate_train=0.266



### نتیجه‌گیری (robust / power / quantile)

- RobustScaler زمانی که داده‌های پرت وجود دارند یک انتخاب پیش‌فرض بسیار خوب است.
- تبدیل‌های توان (Power) وقتی چولگی زیاد است می‌توانند مفید باشند.
- QuantileTransformer گاهی عملکرد بسیار خوبی می‌دهد، اما چون ساختار فاصله‌ها را تغییر می‌دهد باید با cross-validation بررسی شود.

---

## ۸) مقیاس‌بندی و SVM: چرا هایپرپارامترهای «C» و «gamma» به مقیاس وابسته‌اند؟

برای SVM با کرنل RBF داریم:

$$
K(\mathbf{x}, \mathbf{y}) = \exp(-\gamma \lVert \mathbf{x} - \mathbf{y} \rVert_2^2)
$$

اگر ویژگی‌ها را مقیاس‌بندی کنید، فاصله‌ها تغییر می‌کنند و بنابراین معنای «موثر» $\gamma$ نیز تغییر می‌کند.

به همین دلیل **SVM تقریباً همیشه با مقیاس‌بندی استفاده می‌شود** و جست‌وجوی هایپرپارامتر باید با مقیاس‌بندی داخل pipeline انجام شود.

این موضوع را روی `iris` نشان می‌دهیم.


In [10]:

X = iris.drop(columns=["classification"])
y = iris["classification"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

svm_raw = SVC(kernel="rbf", C=3.0, gamma="scale")
svm_scaled = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", C=3.0, gamma="scale"))
])

svm_raw.fit(X_train, y_train)
svm_scaled.fit(X_train, y_train)

pred_raw = svm_raw.predict(X_test)
pred_scaled = svm_scaled.predict(X_test)

print("SVM (no scaling) accuracy:", round(accuracy_score(y_test, pred_raw), 4))
print("SVM (standardized) accuracy:", round(accuracy_score(y_test, pred_scaled), 4))


SVM (no scaling) accuracy: 0.9737
SVM (standardized) accuracy: 0.9474



## ۹) نشت داده (Data Leakage): رایج‌ترین اشتباه در مقیاس‌بندی

**Leakage** زمانی رخ می‌دهد که اطلاعات مجموعه‌ی آزمون به‌نوعی وارد فرآیند آموزش شود.

یک نشت ظریف در مقیاس‌بندی این‌گونه است:

1. `StandardScaler` را روی *کل داده* fit کنید
2. روی train و test transform انجام دهید
3. ارزیابی کنید

این کار اشتباه است؛ چون میانگین و انحراف معیار اسکیلر از داده‌ی آزمون هم اطلاع گرفته است.

تفاوت را بین:

- مقیاس‌بندی غلط (fit روی کل داده)
- مقیاس‌بندی درست (fit فقط روی train) با `Pipeline`

نشان می‌دهیم. مثال را دوباره روی دیتاست diabetes انجام می‌دهیم.


In [11]:

X = diabetes.drop(columns=["classification"])
y = le.fit_transform(diabetes["classification"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# WRONG: fit scaler on all data, then split transformed data
scaler_wrong = StandardScaler()
X_all_scaled = scaler_wrong.fit_transform(X)

Xa_train, Xa_test, ya_train, ya_test = train_test_split(
    X_all_scaled, y, test_size=0.25, random_state=42, stratify=y
)

lr_wrong = LogisticRegression(max_iter=2000)
lr_wrong.fit(Xa_train, ya_train)
pred_wrong = lr_wrong.predict(Xa_test)
acc_wrong = accuracy_score(ya_test, pred_wrong)

# RIGHT: scaler inside pipeline fitted on training only
lr_right = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=2000))
])
lr_right.fit(X_train, y_train)
pred_right = lr_right.predict(X_test)
acc_right = accuracy_score(y_test, pred_right)

print("Accuracy with leakage (wrong):", round(acc_wrong, 4))
print("Accuracy without leakage (right):", round(acc_right, 4))

# Additional sanity check: show that correct scaler stats come ONLY from training
sc = lr_right.named_steps["scaler"]
print("\nCorrect scaler mean (first 3 features):", np.round(sc.mean_[:3], 3))
print("Correct scaler var  (first 3 features):", np.round(sc.var_[:3], 3))


Accuracy with leakage (wrong): 0.7865
Accuracy without leakage (right): 0.7865

Correct scaler mean (first 3 features): [  3.856 121.705  69.559]
Correct scaler var  (first 3 features): [  11.991 1056.913  356.729]



### قانون حرفه‌ای (غیرقابل مذاکره)

اگر قرار است مقیاس‌بندی انجام شود، باید **داخل** فرآیند آموزش fit شود:

- از `Pipeline` استفاده کنید و با cross-validation اعتبارسنجی کنید.
- در محیط عملیاتی (production)، اسکیلر fit‌شده را همراه مدل ذخیره کنید.

---

## ۱۰) انواع داده‌ی ترکیبی: مقیاس‌بندی عددی، کدگذاری دسته‌ای (مثال drug200)

در ML جدولی معمولاً داریم:

- ویژگی‌های عددی که به مقیاس‌بندی نیاز دارند (مثل `Age` و `Na_to_K`)
- ویژگی‌های دسته‌ای که به کدگذاری نیاز دارند (`Sex`، `BP`، `Cholesterol`)

رویکرد درست این است که یک pipeline **ستون‌به‌ستون** بسازید:

- عددی: `SimpleImputer` → `StandardScaler`
- دسته‌ای: `SimpleImputer` → `OneHotEncoder`
- مدل: یک طبقه‌بند (مثلاً رگرسیون لجستیک)

در این مثال برچسب `Drug` چندکلاسه است.


In [12]:

X = drug.drop(columns=["Drug"])
y = drug["Drug"]

numeric_features = ["Age", "Na_to_K"]
categorical_features = ["Sex", "BP", "Cholesterol"]

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_features),
        ("cat", categorical_pipe, categorical_features),
    ]
)

clf = LogisticRegression(max_iter=4000)

pipe = Pipeline([
    ("prep", preprocess),
    ("clf", clf)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print("Accuracy:", round(accuracy_score(y_test, pred), 4))
print("\nClass distribution (test):")
display(pd.Series(y_test).value_counts(normalize=True).to_frame("share").round(3))

print("\nClassification report:\n")
print(classification_report(y_test, pred))


Accuracy: 0.92

Class distribution (test):


,share
Drug,
DrugY,0.46
drugX,0.26
drugA,0.12
drugC,0.08
drugB,0.08



Classification report:

              precision    recall  f1-score   support

       DrugY       0.88      0.96      0.92        23
       drugA       1.00      1.00      1.00         6
       drugB       1.00      0.50      0.67         4
       drugC       1.00      1.00      1.00         4
       drugX       0.92      0.92      0.92        13

    accuracy                           0.92        50
   macro avg       0.96      0.88      0.90        50
weighted avg       0.92      0.92      0.92        50




## ۱۱) ماتریس‌های تنک و مقیاس‌بندی: StandardScaler در برابر MaxAbsScaler

کدگذاری One-Hot معمولاً یک ماتریس طراحی تنک (sparse) ایجاد می‌کند. دو نکته‌ی مهم:

- `StandardScaler(with_mean=True)` را نمی‌توان مستقیم روی ماتریس تنک اعمال کرد (چون مرکز کردن باعث densify شدن می‌شود).
- برای ویژگی‌های تنک، گزینه‌های بهتر:
  - `StandardScaler(with_mean=False)` (تنکی حفظ می‌شود)، یا
  - `MaxAbsScaler` (بر اساس بیشینه‌ی قدر مطلق مقیاس می‌کند و sparsity را حفظ می‌کند).

در این بخش یک ماتریس «عمدتاً تنک» با one-hot می‌سازیم و اسکیلرها را مقایسه می‌کنیم.


In [13]:

from scipy import sparse

toy = drug.sample(120, random_state=0).reset_index(drop=True)
X = toy.drop(columns=["Drug"])

# Force sparse output for compatibility across scikit-learn versions
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

pre_cat_only = ColumnTransformer(
    transformers=[("cat", ohe, ["Sex","BP","Cholesterol"])],
    remainder="drop"
)

X_sparse = pre_cat_only.fit_transform(X)

is_sp = sparse.issparse(X_sparse)
nnz = X_sparse.nnz if is_sp else int(np.count_nonzero(X_sparse))
total = int(X_sparse.shape[0] * X_sparse.shape[1])

print("Transformed shape:", X_sparse.shape)
print("Sparse output:", is_sp)
print("Sparsity (nnz / total):", nnz, "/", total)

# Demonstrate why centering is problematic for sparse matrices
try:
    _ = StandardScaler(with_mean=True).fit_transform(X_sparse)
    print("\nStandardScaler(with_mean=True) succeeded (likely dense input).")
except Exception as e:
    print("\nStandardScaler(with_mean=True) on sparse -> error type:", type(e).__name__)
    print("Message (short):", str(e).splitlines()[0][:140])

X_std_no_mean = StandardScaler(with_mean=False).fit_transform(X_sparse)
X_maxabs = MaxAbsScaler().fit_transform(X_sparse)

print("\nAfter StandardScaler(with_mean=False):")
print("  is_sparse:", sparse.issparse(X_std_no_mean))

print("\nAfter MaxAbsScaler:")
print("  is_sparse:", sparse.issparse(X_maxabs))


Transformed shape: (120, 7)
Sparse output: False
Sparsity (nnz / total): 360 / 840

StandardScaler(with_mean=True) succeeded (likely dense input).

After StandardScaler(with_mean=False):
  is_sparse: False

After MaxAbsScaler:
  is_sparse: False



StandardScaler(with_mean=True) succeeded (likely dense input).

After StandardScaler(with_mean=False):
  is_sparse: False

After MaxAbsScaler:
  is_sparse: False



## ۱۲) مقیاس‌بندی برای PCA و k-means (مثال خوشه‌بندی با hw_200)

PCA و k-means هر دو به هندسه‌ی اقلیدسی وابسته‌اند:

- PCA جهت‌هایی را پیدا می‌کند که واریانس بیشینه دارند. اگر واحد یکی از ویژگی‌ها بزرگ‌تر باشد، مؤلفه‌های اصلی را غالب می‌کند.
- k-means مجموع مربعات درون‌خوشه‌ای را کمینه می‌کند که مستقیماً به مقیاس ویژگی‌ها وابسته است.

از دیتاست `hw_200.csv` (قد/وزن) استفاده می‌کنیم. این فایل عمداً نام ستون‌های نامرتب دارد.

گام‌ها:

1. بارگذاری داده
2. تمیز کردن نام ستون‌ها
3. مقایسه‌ی رفتار PCA و k-means با و بدون مقیاس‌بندی


In [14]:

import re

hw = hw_raw.copy()
print("Original columns:", list(hw.columns))

hw.columns = [c.replace('"', '').strip() for c in hw.columns]
hw.columns = [re.sub(r"\s+", " ", c).strip() for c in hw.columns]
print("Cleaned columns:", list(hw.columns))

display(hw.head())

# robustly pick height/weight columns
hw_cols = [c for c in hw.columns if "Height" in c or "Weight" in c]
X_hw = hw[hw_cols].astype(float).values

print("\nX_hw shape:", X_hw.shape)
print("Feature means (raw):", np.round(X_hw.mean(axis=0), 4))
print("Feature stds  (raw):", np.round(X_hw.std(axis=0), 4))


Original columns: ['Index', ' Height(Inches)"', ' "Weight(Pounds)"']
Cleaned columns: ['Index', 'Height(Inches)', 'Weight(Pounds)']


,Index,Height(Inches),Weight(Pounds)
0,1,65.78,112.99
1,2,71.52,136.49
2,3,69.40,153.03
3,4,68.22,142.34
4,5,67.79,144.30



X_hw shape: (200, 2)
Feature means (raw): [ 67.9498 127.222 ]
Feature stds  (raw): [ 1.9355 11.931 ]


In [15]:

scaler = StandardScaler()

# PCA without scaling
pca_raw = PCA(n_components=2, random_state=0)
Z_raw = pca_raw.fit_transform(X_hw)

# PCA with scaling
X_hw_scaled = scaler.fit_transform(X_hw)
pca_scaled = PCA(n_components=2, random_state=0)
Z_scaled = pca_scaled.fit_transform(X_hw_scaled)

print("Explained variance ratio (PCA raw):", np.round(pca_raw.explained_variance_ratio_, 4))
print("Explained variance ratio (PCA scaled):", np.round(pca_scaled.explained_variance_ratio_, 4))

km_raw = KMeans(n_clusters=3, random_state=0, n_init=10)
labels_raw = km_raw.fit_predict(X_hw)

km_scaled = KMeans(n_clusters=3, random_state=0, n_init=10)
labels_scaled = km_scaled.fit_predict(X_hw_scaled)

print("\nCluster sizes (raw):", np.bincount(labels_raw))
print("Cluster sizes (scaled):", np.bincount(labels_scaled))

centers_raw = km_raw.cluster_centers_
centers_scaled_back = scaler.inverse_transform(km_scaled.cluster_centers_)

centers_df = pd.DataFrame(
    np.vstack([centers_raw, centers_scaled_back]),
    columns=[c.replace("(Inches)", "").replace("(Pounds)", "") for c in hw_cols]
)
centers_df.index = ["raw_c0","raw_c1","raw_c2","scaled_c0_back","scaled_c1_back","scaled_c2_back"]
display(centers_df.round(2))


Explained variance ratio (PCA raw): [0.9825 0.0175]
Explained variance ratio (PCA scaled): [0.7784 0.2216]

Cluster sizes (raw): [93 44 63]
Cluster sizes (scaled): [51 59 90]


,Height,Weight
raw_c0,67.59,126.05
raw_c1,66.66,110.46
raw_c2,69.37,140.66
scaled_c0_back,70.30,139.28
scaled_c1_back,65.99,114.42
scaled_c2_back,67.90,128.78


,Height,Weight
raw_c0,69.50,165.66
raw_c1,66.54,125.63
raw_c2,68.53,144.99
scaled_c0_back,71.38,159.98
scaled_c1_back,66.46,130.81
scaled_c2_back,66.79,155.81



## ۱۳) مقیاس‌بندی و منظم‌سازی: چرا Ridge به واحدها وابسته است؟

رگرسیون Ridge:

$$
\min_{\mathbf{w}} \; \frac{1}{n}\sum_{i=1}^n (y_i - \mathbf{w}^\top \mathbf{x}_i)^2 + \lambda \lVert \mathbf{w} \rVert_2^2
$$

مقیاس‌بندی باعث می‌شود ترم منظم‌سازی بین ویژگی‌ها «منصفانه‌تر» عمل کند.

### مثال: Ridge regression روی house prices

یک pipeline با داده‌ی ترکیبی می‌سازیم (مقیاس‌بندی عددی + one-hot دسته‌ای) و Ridge را با و بدون مقیاس‌بندی عددی مقایسه می‌کنیم.


In [16]:

house = pd.read_csv(Path("../../../Datasets/Regression/house-prices.csv"))
X = house.drop(columns=["Price"])
y = house["Price"].astype(float)

numeric_features = ["SqFt","Bedrooms","Bathrooms","Offers"]
categorical_features = ["Brick","Neighborhood"]

numeric_no_scale = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

numeric_scaled = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

pre_A = ColumnTransformer([
    ("num", numeric_no_scale, numeric_features),
    ("cat", cat_pipe, categorical_features)
])

pre_B = ColumnTransformer([
    ("num", numeric_scaled, numeric_features),
    ("cat", cat_pipe, categorical_features)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# RMSE helper (compatible across scikit-learn versions)
try:
    from sklearn.metrics import root_mean_squared_error
    def rmse(y_true, y_pred):
        return root_mean_squared_error(y_true, y_pred)
except Exception:
    from sklearn.metrics import mean_squared_error
    import numpy as np
    def rmse(y_true, y_pred):
        return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# (A) Same alpha for both pipelines: shows alpha depends on feature scale
alpha_fixed = 10.0
pipe_A_fixed = Pipeline([("prep", pre_A), ("model", Ridge(alpha=alpha_fixed, random_state=0))])
pipe_B_fixed = Pipeline([("prep", pre_B), ("model", Ridge(alpha=alpha_fixed, random_state=0))])

pipe_A_fixed.fit(X_train, y_train)
pipe_B_fixed.fit(X_train, y_train)

pred_A = pipe_A_fixed.predict(X_test)
pred_B = pipe_B_fixed.predict(X_test)

rmse_A = rmse(y_test, pred_A)
rmse_B = rmse(y_test, pred_B)
r2_A = r2_score(y_test, pred_A)
r2_B = r2_score(y_test, pred_B)

print(f"Fixed alpha={alpha_fixed}")
print("  Ridge (no numeric scaling):  RMSE =", round(rmse_A, 2), "  R2 =", round(r2_A, 4))
print("  Ridge (with scaling):       RMSE =", round(rmse_B, 2), "  R2 =", round(r2_B, 4))

# (B) Fair comparison: tune alpha separately
alphas = np.logspace(-3, 4, 12)

def tune_ridge(preprocessor, name):
    pipe = Pipeline([("prep", preprocessor), ("model", Ridge(random_state=0))])
    gs = GridSearchCV(pipe, {"model__alpha": alphas}, cv=5, scoring="neg_root_mean_squared_error")
    gs.fit(X_train, y_train)
    best = gs.best_estimator_
    pred = best.predict(X_test)
    return {
        "name": name,
        "best_alpha": gs.best_params_["model__alpha"],
        "test_rmse": rmse(y_test, pred),
        "test_r2": r2_score(y_test, pred),
        "cv_rmse": -gs.best_score_,
    }

res_A = tune_ridge(pre_A, "No scaling (tuned alpha)")
res_B = tune_ridge(pre_B, "Scaled numeric (tuned alpha)")

print("\nAfter tuning alpha (fair comparison):")
for r in [res_A, res_B]:
    print(f"  {r['name']}: best_alpha={r['best_alpha']:.4g}  CV_RMSE={r['cv_rmse']:.2f}  Test_RMSE={r['test_rmse']:.2f}  Test_R2={r['test_r2']:.4f}")


Fixed alpha=10.0
  Ridge (no numeric scaling):  RMSE = 11290.03   R2 = 0.7921
  Ridge (with scaling):       RMSE = 10347.16   R2 = 0.8253

After tuning alpha (fair comparison):
  No scaling (tuned alpha): best_alpha=0.3511  CV_RMSE=9992.95  Test_RMSE=10334.16  Test_R2=0.8258
  Scaled numeric (tuned alpha): best_alpha=0.3511  CV_RMSE=9992.66  Test_RMSE=10291.81  Test_R2=0.8272



## ۱۴) اسکیلر به‌عنوان هایپرپارامتر: یک Grid Search کوچک (Iris + kNN)

در سطح پیشرفته، بهتر است *انتخاب اسکیلر* را اعتبارسنجی کنید و فرض نکنید که همیشه یک گزینه بهترین است.

در این مثال مقایسه می‌کنیم:

- بدون اسکیلر
- min–max
- استانداردسازی
- robust scaling

یک `GridSearchCV` کوچک انجام می‌دهیم که در آن اسکیلر بخشی از pipeline است.

این الگو در بسیاری از مدل‌ها قابل استفاده است.


In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message=".*invalid value encountered in cast.*")


X = iris.drop(columns=["classification"])
y = iris["classification"]

pipe = Pipeline([
    ("scaler", "passthrough"),
    ("knn", KNeighborsClassifier())
])

param_grid = {
    "scaler": ["passthrough", StandardScaler(), MinMaxScaler(), RobustScaler()],
    "knn__n_neighbors": [3,5,7,9,11],
    "knn__weights": ["uniform", "distance"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
gs = GridSearchCV(pipe, param_grid=param_grid, cv=cv, scoring="accuracy", n_jobs=None)
gs.fit(X, y)

print("Best CV accuracy:", round(gs.best_score_, 4))
print("Best params:")
for k, v in gs.best_params_.items():
    print(" ", k, "=", v)


Best CV accuracy: 0.9733
Best params:
  knn__n_neighbors = 11
  knn__weights = uniform
  scaler = passthrough



## ۱۵) انتخاب اسکیلر: یک جدول تصمیم‌گیری عملی

هیچ اسکیلر «بهترینِ مطلق» وجود ندارد. انتخاب را بر اساس موارد زیر انجام دهید:

### حساسیت الگوریتم
- kNN، k-means، SVM، PCA: مقیاس‌بندی معمولاً ضروری است.
- رگرسیون لجستیک/خطی با منظم‌سازی: مقیاس‌بندی به‌شدت توصیه می‌شود.
- مدل‌های درختی: اغلب لازم نیست، اما می‌تواند جزئی از یک pipeline یکپارچه باشد.

### توزیع داده
- تقریباً متقارن، پرت‌های کم → `StandardScaler`
- پرت/دُم‌سنگین → `RobustScaler`
- نیاز به بازه‌ی محدود → `MinMaxScaler`
- ویژگی‌های تنک → `MaxAbsScaler` یا `StandardScaler(with_mean=False)`
- چولگی زیاد → `PowerTransformer` یا `QuantileTransformer` (حتماً اعتبارسنجی شود)

### حاکمیت مدل و استقرار
- اسکیلر فقط باید روی داده‌ی آموزش fit شود.
- پیش‌پردازش + مدل را به‌صورت یک آرتیفکت واحد (pipeline) ذخیره کنید.
- انتخاب اسکیلر را در کنار مدل مستند کنید.

---

## ۱۶) تمرین‌ها (پیشنهادی)

1. روی دیتاست `diabetes`، `StandardScaler` و `RobustScaler` را برای `LogisticRegression` با CV پنج‌تایی مقایسه کنید.
2. روی `wine`، یک SVM با و بدون مقیاس‌بندی اجرا کنید و حساسیت به `gamma` را مشاهده کنید.
3. روی `hw_200`، `MinMaxScaler` و `StandardScaler` را برای k-means امتحان کنید و مراکز خوشه‌ها را مقایسه کنید.
4. یک جست‌وجوی کوچک هایپرپارامتر بسازید که خودِ اسکیلر هم بخشی از فضای جست‌وجو باشد.

---

### جمع‌بندی

مقیاس‌بندی ظاهری نیست. مقیاس‌بندی ساختار متریک را تعریف می‌کند، منصفانه بودن منظم‌سازی را کنترل می‌کند، و می‌تواند از ناپایداری عددی جلوگیری کند. از pipeline استفاده کنید، از نشت داده پرهیز کنید، و انتخاب اسکیلر را مثل هر هایپرپارامتر دیگر اعتبارسنجی کنید.
